<a href="https://colab.research.google.com/github/krystaldowling/MAST30034_Final_Project-/blob/machine_learning_doc2vec/classifier_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import scipy
import pickle

from sklearn.model_selection import train_test_split

news_data = pd.read_csv("/content/drive/My Drive/Data/final_preproccessed_data.csv")

news_data.index.name = 'index'

news_train, news_test, label_train, label_test = train_test_split(news_data.loc[:,['title','text']], 
                                                        news_data.loc[:,'label'], test_size=0.33, 
                                                        random_state=88)

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim
from gensim.test.utils import get_tmpfile

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

fname = get_tmpfile("my_doc2vec_model")


model.save(fname)
model = Doc2Vec.load(fname)  # you can continue training with the loaded model!
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
vector = model.infer_vector(["system", "response","hello"])
print(vector)

df = news_train

def doc2vec_fun(x):
    L = []
    for elem in x.split(' '):
        elem = (elem.replace('\n', '').replace(',', '').replace('.', '').
        replace('”', '').replace('?', '').replace('\'', '').lower().
        replace('“', '').replace(']', '').replace('[', ''))
        L.append(elem)
    vector = model.infer_vector(L)
    return vector
df['doc2vec_title']=df['title'].map(lambda x:doc2vec_fun(x))
df['doc2vec_text']=df['text'].map(lambda x:doc2vec_fun(x))

print(df.head(10))

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[-0.03556247 -0.09903148  0.07664264  0.09468351 -0.02956764]
                                                   title  ...                                       doc2vec_text
index                                                     ...                                                   
20183  Policy Shift Helps Coal, but Other Forces May ...  ...  [0.0734534, 0.0027341133, -0.01355525, -0.0641...
1750   Dr. David Duke and Adrian Salbuch : The intern...  ...  [0.03029924, -0.034626447, -0.008349692, -0.03...
1710   Donald Trump Jr. Met With Russian Advocates Ab...  ...  [-0.03895306, -0.094609015, -0.0064553455, 0.0...
3801             Why Trump Won and Why Clinton Lost    :  ...  [-0.06605714, -0.04280777, 0.03766791, -0.0653...
24095  The Battle of ‘Miss Saigon’: Yellowface, Art a...  ...  [0.053424653, 0.038532283, 0.012829526, 0.0366...
23109  Trump Meets With Some Vocal Critics: Editors a...  ...  [0.026124837, -0.029834874, -0.08105983, -0.09...
12983  YouTube Advertiser Exodus H

In [ ]:
df_test = news_test

df_test['doc2vec_title']=df_test['title'].map(lambda x:doc2vec_fun(x))
df_test['doc2vec_text']=df_test['text'].map(lambda x:doc2vec_fun(x))

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.svm import SVC

def train_models(X_train, X_test, y_train, y_test):
    '''
    function performs feature selection and fits Gaussian Naive Bayes, 
    Multinomial Naive Bayes, Logistic Regression and Support Vector Classification
    on training data - finds the accuracy score of the model on the test data
    '''
  

    models = [GaussianNB(),
            MultinomialNB(),
            SVC(),
            LogisticRegression()]

    titles = ['GNB',
            'MNB',
            'SVC',
            'Logistic Regression']

    k = 10

    x2 = SelectKBest(chi2, k)
    x2.fit(X_train,y_train)
    X_train_x2 = x2.transform(X_train)
    X_test_x2 = x2.transform(X_test)


    mi = SelectKBest(mutual_info_classif, k)
    mi.fit(X_train,y_train)
    X_train_mi = mi.transform(X_train)
    X_test_mi = mi.transform(X_test)


    Xs = [(X_train_x2, X_test_x2), (X_train_mi, X_test_mi)]
     
  
    X_names = ['x2', 'mi']
    

    for title, model in zip(titles, models):
        print('\n',title, '(with k=',k,'features):')
        for X_name, X in zip(X_names, Xs):
            X_train_t, X_test_t = X
            model.fit(X_train_t.todense(), y_train)
            acc = model.score(X_test_t.todense(), y_test)
            print(X_name, '\t acc',  acc)

    return

train_models(df['doc2vec_text'], df_test['doc2vec_text'], label_train, label_test)

ValueError: ignored